In [2]:
!pip install unsloth trl peft accelerate bitsandbytes

In [3]:
from datasets import load_dataset

ds = load_dataset("gretelai/symptom_to_diagnosis")
print(ds.keys())
print(len(ds["train"]))
print(ds["train"][0])

import json

# Load the training split of the dataset into a list of dictionaries
file = ds["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dict_keys(['train', 'test'])
853
{'output_text': 'cervical spondylosis', 'input_text': "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."}


In [4]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [5]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/mistral-7b-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

In [5]:
# from datasets import Dataset

# print(ds['train'][0])

# def format_prompt(example):
#     return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

# formatted_data = [format_prompt(item) for item in file]
# dataset = Dataset.from_dict({"text": formatted_data})

In [6]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.8.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
def reformat_data(example):
  """Reformats a dataset example into the desired JSON structure."""
  return {
      "instruction": example["input_text"],
      "input": "",
      "output": example["output_text"]
  }

# Apply the reformatting function to the training split and remove original columns
reformatted_train_data = ds["train"].map(reformat_data, remove_columns=["input_text", "output_text"])

# Display the first reformatted example to verify
print(reformatted_train_data[2])

{'instruction': 'I have been urinating blood. I sometimes feel sick to my stomach when I urinate. I often feel like I have a fever.', 'input': '', 'output': 'urinary tract infection'}


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Define the formatting function
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Apply a standard instruction tuning template
        text = f"### Instruction:\n{instruction}\n### Input:\n{input}\n### Output:\n{output}<|endoftext|>"
        texts.append(text)
    # Workaround: Return a dummy string if the list is empty to prevent IndexError during trainer validation
    if not texts:
        return ["<dummy_text>"]
    return texts # Return the list of strings


# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=reformatted_train_data,
    formatting_func=formatting_prompts_func, # Pass the formatting function here
    max_seq_length=max_seq_length,
    dataset_num_proc=1, # Changed from 2 to 1
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

In [9]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 853 | Num Epochs = 3 | Total steps = 321
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 7,409,504,256 (2.26% trained)


Step,Training Loss,entropy
25,1.145300,0
50,0.974100,No Log
75,0.871900,No Log
100,0.772600,No Log
125,0.606600,No Log
150,0.564000,No Log
175,0.570400,No Log
200,0.528800,No Log
225,0.409300,No Log
250,0.312700,No Log


In [10]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# User instruction input
user_instruction = "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."

# Format the input according to the training template
# We only provide the instruction and input fields for inference
prompt = f"### Instruction:\n{user_instruction}\n### Input:\n"

# Tokenize the formatted prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print the full response
response = tokenizer.batch_decode(outputs)[0]
print(response)

# Optionally, extract just the generated output part
# Assuming the model generates "### Output:\n..." after the prompt
output_start_index = response.find("### Output:\n")
if output_start_index != -1:
    generated_output = response[output_start_index + len("### Output:\n"):].strip()
    print("\nGenerated Output:")
    print(generated_output)
else:
    print("\nCould not find '### Output:' in the generated response.")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> ### Instruction:
I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak.
### Input:

### Output:
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
### Output:
cervical spondylosis<|endoftext|>
cervical spondylosis

Generated Output:
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|endoftext|>
cervical spondylosis<|en

In [11]:
test=ds['test']

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 4.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.87 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 53%|█████▎    | 17/32 [00:01<00:01, 14.84it/s]
We will save to Disk and not RAM now.
 97%|█████████▋| 31/32 [01:39<00:07,  7.17s/it]

NameError: name 'protoc' is not defined

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

# Task
Evaluate the fine-tuned model on the test data (`ds["test"]`) by comparing the model's predicted diagnoses with the true diagnoses.

## Load test data

### Subtask:
Load the test split of the dataset (`ds["test"]`).


**Reasoning**:
Access the test split of the dataset and store it in a variable, then print its keys and length to verify.



In [ ]:
test_data = ds["test"]
print(test_data.features)
print(len(test_data))

## Reformat test data

### Subtask:
Apply the same reformatting function used for training data to the test data to get it into the `instruction`, `input`, `output` dictionary format.


**Reasoning**:
Apply the reformatting function to the test data and remove the original columns.



In [ ]:
reformatted_test_data = test_data.map(reformat_data, remove_columns=["input_text", "output_text"])
print(reformatted_test_data[0])

## Prepare for inference

### Subtask:
Iterate through the reformatted test data. For each example, create a prompt string using the same template as used for training and inference (`### Instruction:
{instruction}
### Input:
`).


**Reasoning**:
Initialize an empty list and iterate through the reformatted test data to create and append the formatted prompt strings to the list.



In [ ]:
prompts = []
for example in reformatted_test_data:
    instruction = example["instruction"]
    input_text = example["input"]
    prompt = f"### Instruction:\n{instruction}\n### Input:\n{input_text}"
    prompts.append(prompt)

print(prompts[0])

## Run inference

### Subtask:
For each prepared prompt, run inference using the fine-tuned model to generate the predicted diagnosis.


**Reasoning**:
Iterate through the prepared prompts, tokenize each one, generate a response using the fine-tuned model, decode the response, and store it in a list.



In [ ]:
predicted_outputs = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        input_ids=inputs.input_ids,
        max_new_tokens=256,
        use_cache=True,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )
    response = tokenizer.batch_decode(outputs)[0]
    predicted_outputs.append(response)

print("First few predicted outputs:")
for i in range(min(5, len(predicted_outputs))):
    print(f"--- Output {i+1} ---")
    print(predicted_outputs[i])

**Reasoning**:
The model has generated responses for each prompt. The next step is to extract the actual predicted diagnosis from these generated strings, as the raw output includes the input prompt and extra tokens.



In [ ]:
extracted_predictions = []
for response in predicted_outputs:
    # Find the start of the '### Output:' section
    output_start_index = response.find("### Output:\n")
    if output_start_index != -1:
        # Extract the part after '### Output:'
        generated_output = response[output_start_index + len("### Output:\n"):].strip()
        # Remove the '<|endoftext|>' token if present
        if generated_output.endswith("<|endoftext|>"):
            generated_output = generated_output[:-len("<|endoftext|>")].strip()
        # Some responses might contain repeated outputs or extra text after the first prediction
        # We'll try to take the first distinct diagnosis if there are multiple
        # Split by common separators like newline or the template pattern itself
        parts = generated_output.split("### Output:")
        cleaned_output = parts[0].strip()
        if "<|endoftext|>" in cleaned_output:
             cleaned_output = cleaned_output.split("<|endoftext|>")[0].strip()

        extracted_predictions.append(cleaned_output)
    else:
        # If '### Output:' is not found, append a placeholder or the raw response
        extracted_predictions.append("Extraction Failed")

print("First few extracted predictions:")
for i in range(min(5, len(extracted_predictions))):
    print(f"--- Prediction {i+1} ---")
    print(extracted_predictions[i])


**Reasoning**:
The predicted diagnoses have been extracted. The next step is to get the true diagnoses from the reformatted test data to compare them.



In [ ]:
true_diagnoses = [example["output"] for example in reformatted_test_data]

print("First few true diagnoses:")
for i in range(min(5, len(true_diagnoses))):
    print(f"--- True Diagnosis {i+1} ---")
    print(true_diagnoses[i])

## Evaluate model

### Subtask:
Compare the list of generated outputs with the list of true outputs to calculate evaluation metrics such as accuracy.


**Reasoning**:
Calculate the accuracy of the model's predictions by comparing the extracted predictions with the true diagnoses and print the result.



In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(true_diagnoses, extracted_predictions)
print(f"Accuracy: {accuracy}")

## Display results

### Subtask:
Print or display the evaluation results.


**Reasoning**:
Print the calculated accuracy score and the number of test examples evaluated as requested by the subtask instructions.



In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Number of test examples evaluated: {len(true_diagnoses)}")

## Summary:

### Data Analysis Key Findings

*   The fine-tuned model achieved an accuracy of approximately 0.9764 when predicting diagnoses on the test dataset.
*   The evaluation was conducted on 212 test examples.

### Insights or Next Steps

*   The high accuracy suggests the fine-tuned model is performing very well on this specific diagnostic task.
*   Further analysis could involve examining the specific cases where the model made incorrect predictions to identify potential areas for improvement or limitations.


In [ ]:
# Analyze the dataset for repeated data and unique output texts
import pandas as pd

# Convert the dataset to a pandas DataFrame for easier analysis
df_train = pd.DataFrame(ds["train"])
df_test = pd.DataFrame(ds["test"])

# Combine train and test for overall analysis if needed, or analyze separately
# For checking repeated data across splits, combine:
df_combined = pd.concat([df_train, df_test])

unique_output_texts_test= df_test['output_text'].nunique()
print(f"Number of repeated data entries across test splits: {unique_output_texts_test}")

# Find repeated data (checking for duplicate rows across both input and output text)
repeated_data_count = df_combined.duplicated().sum()
print(f"Number of repeated data entries across train and test splits: {repeated_data_count}")

# Find unique output text (diagnoses) in the entire dataset
unique_output_texts = df_combined['output_text'].nunique()
print(f"Number of unique output texts (diagnoses) in the dataset: {unique_output_texts}")

# Also check within training data specifically for context
repeated_data_train_count = df_train.duplicated().sum()
print(f"Number of repeated data entries in the training split: {repeated_data_train_count}")
unique_output_texts_train = df_train['output_text'].nunique()
print(f"Number of unique output texts (diagnoses) in the training split: {unique_output_texts_train}")

# And test data
repeated_data_test_count = df_test.duplicated().sum()
print(f"Number of repeated data entries in the test split: {repeated_data_test_count}")
unique_output_texts_test = df_test['output_text'].nunique()
print(f"Number of unique output texts (diagnoses) in the test split: {unique_output_texts_test}")

### Dataset Analysis Summary

*   **Repeated Data**: The dataset has **{repeated_data_count}** repeated data entries across the training and test splits. Within the training split there are **{repeated_data_train_count}** repeated entries and within the test split there are **{repeated_data_test_count}** repeated entries.
*   **Unique Output Texts (Diagnoses)**: There are **{unique_output_texts}** unique diagnoses in the entire dataset. The training split contains **{unique_output_texts_train}** unique diagnoses, and the test split contains **{unique_output_texts_test}** unique diagnoses.

This dataset appears to be a collection of symptom descriptions paired with a corresponding medical diagnosis. It is structured for tasks like training a model to predict a diagnosis based on provided symptoms. The presence of repeated data in the training set could potentially influence model training depending on the training strategy, while the number of unique diagnoses indicates the variety of conditions the model is expected to predict.

In [15]:
!pip uninstall protobuf -y
!pip install protobuf==3.20.3

Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
